In [65]:
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
import numpy as np
import nltk
import spacy
import spacy.symbols as ss
from spacy import displacy
import en_core_web_lg
import pandas as pd
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 300)

In [2]:
nlp = en_core_web_lg.load()

In [61]:
df = pd.read_table("../../../../../tasks/02-structural-linguistics/blog2008.txt", names=['text'])[:1000]

In [62]:
df = df.assign(doc=df.text.apply(nlp))

In [20]:
df.iloc[176]

text                              The government still is n't saying definitively but it would appear that they are giving credence to reports that Bhutto was shot to death .
doc     (The, government, still, is, n't, saying, definitively, but, it, would, appear, that, they, are, giving, credence, to, reports, that, Bhutto, was, shot, to, death, .)
Name: 176, dtype: object

In [82]:
d = df.iloc[8]['doc']
say_syns = set([
    "say", "tell", "speak", "claim", "communicate", "talk",
    "explain", "verbalize", "chat", "communicate", "describe",
    "express", "advertise"])
def find_collocations(doc, say_syns=say_syns):
    collocations = []
    for tok in doc:
        if tok.lemma_ in say_syns and tok.pos == ss.VERB:
            for verb_child in tok.children:
                if verb_child.pos == ss.ADV and verb_child.text.endswith('ly'):
                    collocations.append((tok.lemma_, verb_child.lemma_))
                    for adv_child in verb_child.children:
                        if adv_child.dep == ss.conj:
                            collocations.append((tok.lemma_, adv_child.lemma_))
    return collocations

In [83]:
df = df.assign(collocations = df['doc'].apply(find_collocations))

In [84]:
df[df['collocations'].apply(lambda x: len(x) != 0)]

,text,doc,collocations
40,The electoral commission initially claimed that roughly a quarter of returning officers disappeared for 36 hours without announcing results and had switched off their mobile phones .,"(The, electoral, commission, initially, claimed, that, roughly, a, quarter, of, returning, officers, disappeared, for, 36, hours, without, announcing, results, and, had, switched, off, their, mobile, phones, .)","[(claim, initially)]"
176,The government still is n't saying definitively but it would appear that they are giving credence to reports that Bhutto was shot to death .,"(The, government, still, is, n't, saying, definitively, but, it, would, appear, that, they, are, giving, credence, to, reports, that, Bhutto, was, shot, to, death, .)","[(say, definitively)]"
264,These pages recently said goodbye to global warming .,"(These, pages, recently, said, goodbye, to, global, warming, .)","[(say, recently)]"
725,"Not overtly and not directly , but she will speak in code saying that Obama ca n't win .","(Not, overtly, and, not, directly, ,, but, she, will, speak, in, code, saying, that, Obama, ca, n't, win, .)","[(speak, overtly), (speak, directly)]"


In [42]:
displacy.render(nlp(df.text[8]), jupyter=True)